# Match searches with bookings

In [1]:
import pandas as pd
# Load data from csv file
chunksize = 10 ** 6
path = "../data/searches.csv"
delim = "^"

chunks_s = pd.read_csv(path, chunksize = chunksize, sep = delim, header = 0,
                       parse_dates = ['Date'],
                       usecols = ['Date', 'Origin', 'Destination', 'OfficeID'])
searches = pd.DataFrame()
searches = pd.concat(chunks_s)

In [2]:
path_b = "../data/bookings.csv"
chunks_b = pd.read_csv(path_b, chunksize = chunksize, sep = delim, header = 0,
                         parse_dates = ['act_date           '],
                         usecols = ['act_date           ', 'dep_port', 'arr_port', 'pos_oid  '])
bookings = pd.DataFrame()
bookings = pd.concat(chunks_b)

In [3]:
# print data
print searches[:5]
print bookings[:5]

                  Date                          OfficeID Origin Destination
0  2013-01-01 00:00:00  624d8c3ac0b3a7ca03e3c167e0f48327    TXL         AUH
1  2013-01-01 00:00:00  b0af35b31588dc4ab06d5cf2986e8e02    ATH         MIL
2  2013-01-01 00:00:00  3561a60621de06ab1badc8ca55699ef3    ICT         SFO
3  2013-01-01 00:00:00  1864e5e8013d9414150e91d26b6a558b    RNB         ARN
4  2013-01-01 00:00:00  1ec336348f44207d2e0027dc3a68c118    OSL         MAD
  act_date                                    pos_oid    dep_port  arr_port
0          2013-03-05  1a11ae49fcbf545fd2afc1a24d88d2b7  ZRH       LHR     
1          2013-03-26  7437560d8f276d6d05eeb806d9e7edee  SAL       CLT     
2          2013-03-26  7437560d8f276d6d05eeb806d9e7edee  SAL       CLT     
3          2013-03-26  36472c6dbaf7afec9136ac40364e2794  AKL       SVO     
4          2013-03-26  36472c6dbaf7afec9136ac40364e2794  AKL       SVO     


In [4]:
# primary data processing (remove whitespaces in headers and values, rename headers)
# secondary data processing (rename columns in bookings in order to fit the same names in searches)
bookings.rename(columns={'act_date           ' : 'Date', 'dep_port' : 'Origin',
                         'arr_port' : 'Destination', 'pos_oid  ' : 'OfficeID'}, inplace = True)
bookings["OfficeID"] = bookings["OfficeID"].map(str.strip)
bookings["Origin"] = bookings["Origin"].map(str.strip)
bookings["Destination"] = bookings["Destination"].map(str.strip)
# secondary data processing (extract only date)
bookings['Date'] = bookings["Date"].astype(str)
searches['Date'] = searches["Date"].astype(str)
bookings['Date'] = bookings.apply(lambda row: row['Date'][:10], axis=1)
searches['Date'] = searches.apply(lambda row: row['Date'][:10], axis=1)

In [5]:
# create a column called booked and assign 1 to it
bookings['booked'] = 1 
bookings[:5]

,Date,OfficeID,Origin,Destination,booked
0,2013-03-05,1a11ae49fcbf545fd2afc1a24d88d2b7,ZRH,LHR,1
1,2013-03-26,7437560d8f276d6d05eeb806d9e7edee,SAL,CLT,1
2,2013-03-26,7437560d8f276d6d05eeb806d9e7edee,SAL,CLT,1
3,2013-03-26,36472c6dbaf7afec9136ac40364e2794,AKL,SVO,1
4,2013-03-26,36472c6dbaf7afec9136ac40364e2794,AKL,SVO,1


In [6]:
# left join two tables 'searches' and 'bookings'
result = pd.merge(searches, bookings, how = 'left', on = [ 'Origin', 'Destination', 'OfficeID', 'Date'])

result

,Date,OfficeID,Origin,Destination,booked
0,2013-01-01,624d8c3ac0b3a7ca03e3c167e0f48327,TXL,AUH,NaN
1,2013-01-01,b0af35b31588dc4ab06d5cf2986e8e02,ATH,MIL,NaN
2,2013-01-01,3561a60621de06ab1badc8ca55699ef3,ICT,SFO,NaN
3,2013-01-01,1864e5e8013d9414150e91d26b6a558b,RNB,ARN,NaN
4,2013-01-01,1ec336348f44207d2e0027dc3a68c118,OSL,MAD,NaN
5,2013-01-01,3561a60621de06ab1badc8ca55699ef3,IAH,BLR,NaN
6,2013-01-01,d327ca6e35cc6732d4709828327ac7c1,CPH,PAR,NaN
7,2013-01-01,38a3abb0a28e3f00fa79a11f552a5052,PAR,DUB,NaN
8,2013-01-01,c8daef4f8bf73a61aa2c928705f7b82d,DUS,ACE,NaN
9,2013-01-01,28d7a8c95e4db88589d3d35b66920e78,FRA,BGW,NaN


In [11]:
# replace NAN in the column 'booked' by 0
result['booked'].fillna(0, inplace=True)
result[:5]

,Date,OfficeID,Origin,Destination,booked
0,2013-01-01,624d8c3ac0b3a7ca03e3c167e0f48327,TXL,AUH,0
1,2013-01-01,b0af35b31588dc4ab06d5cf2986e8e02,ATH,MIL,0
2,2013-01-01,3561a60621de06ab1badc8ca55699ef3,ICT,SFO,0
3,2013-01-01,1864e5e8013d9414150e91d26b6a558b,RNB,ARN,0
4,2013-01-01,1ec336348f44207d2e0027dc3a68c118,OSL,MAD,0


In [8]:
# display some booked searches
result.loc[result['booked'] == 1][:5]

,Date,OfficeID,Origin,Destination,booked
923,2013-01-01,cf818ddafad144a83967003057e64033,DEL,BOM,1
924,2013-01-01,cf818ddafad144a83967003057e64033,DEL,BOM,1
925,2013-01-01,cf818ddafad144a83967003057e64033,DEL,BOM,1
926,2013-01-01,cf818ddafad144a83967003057e64033,DEL,BOM,1
927,2013-01-01,cf818ddafad144a83967003057e64033,DEL,BOM,1


In [12]:
## write to a csv file
result.to_csv('../data/searches_new.csv', sep = "^", index = False, encoding = "utf-8")

In [13]:
result

,Date,OfficeID,Origin,Destination,booked
0,2013-01-01,624d8c3ac0b3a7ca03e3c167e0f48327,TXL,AUH,0
1,2013-01-01,b0af35b31588dc4ab06d5cf2986e8e02,ATH,MIL,0
2,2013-01-01,3561a60621de06ab1badc8ca55699ef3,ICT,SFO,0
3,2013-01-01,1864e5e8013d9414150e91d26b6a558b,RNB,ARN,0
4,2013-01-01,1ec336348f44207d2e0027dc3a68c118,OSL,MAD,0
5,2013-01-01,3561a60621de06ab1badc8ca55699ef3,IAH,BLR,0
6,2013-01-01,d327ca6e35cc6732d4709828327ac7c1,CPH,PAR,0
7,2013-01-01,38a3abb0a28e3f00fa79a11f552a5052,PAR,DUB,0
8,2013-01-01,c8daef4f8bf73a61aa2c928705f7b82d,DUS,ACE,0
9,2013-01-01,28d7a8c95e4db88589d3d35b66920e78,FRA,BGW,0
